In [11]:
from ipynb.fs.full.feature_engineering_catboost import X_Transformer_scaled
from ipynb.fs.full.feature_engineering_catboost import X_Transformer
from ipynb.fs.full.feature_engineering_catboost import DataLoader
from ipynb.fs.full.feature_engineering_catboost import TargetNormalizedRegressor
from lightgbm import LGBMRegressor
from ipynb.fs.full.feature_engineering_final import BlendingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV
import xgboost as xgb
import numpy as np
import pandas as pd
from xgboost import plot_tree
from mlxtend.regressor import StackingCVRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_tree
from matplotlib.pylab import rcParams
from sklearn.model_selection import KFold
from sklearn.svm import SVR

In [12]:
data_loader = DataLoader("train.csv","test.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [13]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()
X_test = data_loader.getX_test()

In [14]:
x_transformer = X_Transformer()

In [15]:
kf = KFold(n_splits=10,shuffle = True,random_state = 42)

In [16]:
lgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=4700,
                                       max_bin=200, 
                                       bagging_fraction=0.85,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.3,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       categorical_feature = "auto",
                                       max_cat_to_onehot = 4,
                                       min_data_per_group = 24,  
                                       cat_l2 = 0                               
                                       )

lgbm_t = TargetNormalizedRegressor(lgbm)


In [17]:
learning_pipeline = Pipeline(steps=[('preprocess', x_transformer),('learn',lgbm_t)])
learning_pipeline.fit(X_train,y=Y_train)





/home/victor/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:741: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))
/home/victor/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=None,
                          steps=[('col_drop',
                                  ColumnDrop(columns=['Utilities', 'Street',
                                                      'PoolQC'])),
                                 ('col_transformer',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='passthrough',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('categorical_with_missing_values_const',
                                                                   PipelineWithFeatureNames(feature_names=['...
                                                                   learning_rate=0.01,
                                                                

In [18]:
#set up the parameters
#rcParams['figure.figsize'] = 100,90
#plot_tree(learning_pipeline.named_steps['learn'],num_trees=2799, rankdir='LR')#plt.show()


In [19]:
Ypredicted = learning_pipeline.predict(X_test)
#Ypredicted =   #pd.read_csv('submission_stack_elasticnet.csv').iloc[:,1]
#Ypredicted = pd.read_csv('submission_stack_ridge.csv').iloc[:,1]*0.5 + pd.read_csv('submission_stack_elasticnet.csv').iloc[:,1]*0.5

In [20]:
resultDf = pd.DataFrame()
resultDf['Id'] = data_loader.get_Test_id()
resultDf['SalePrice'] = Ypredicted

#q1 = resultDf['SalePrice'].quantile(0.0045)
#q2 = resultDf['SalePrice'].quantile(0.99)
#resultDf[resultDf['SalePrice'] <= q1]
#resultDf[resultDf['SalePrice'] <= q1]

#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x > q1 else x*0.9)
#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x < q2 else x*1.081)

#resultDf
resultDf.to_csv('submission.csv',index = False)